In [1]:
import os

from tqdm import tqdm

import pandas as pd
import numpy as np

from run_kmeans import RunKmeans

from os.path import join
from numpy import min, mean, sum

In [2]:
pwd = os.getcwd()
datadir = join(pwd, 'data/')
files = os.listdir(datadir)
filename = 'boston.csv'
file = join(datadir, filename)
num_clusters = [
    5,
]
inits = [
    'random',
    'k-means++',
    'coc',
    'orss',
]

In [3]:
for file in tqdm(files[:]):
    filepath = join(datadir, file)
    comparator = RunKmeans(
        n_clusters=5,
        num_exp=20,
        file=filepath,
    )
    inertias, times, iters = comparator.run_kmeans()
    print(file)
    for k in inertias:
        for inertia in inertias[k]:
            if type(inertia) == type(np.float64(1.0)):
                continue
            elif type(inertia) == type(np.int64(1.0)):
                continue
            else:
                raise TypeError(inertia, 'not float or int')
        print(k, mean(inertias[k]), mean(times[k]))

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

airlines.csv
random 5788610179951.841 1.7710633150000006
k-means++ 5875490350040.696 1.1635343500000002
orss 5873660965260.748 1.3813285949999994
coc 5804937590275.444 1.3317915850000006


 11%|█████████▏                                                                         | 1/9 [01:52<15:03, 112.97s/it]

boston.csv
random 2555845.207785929 0.06625820000000004
k-means++ 1534215.3203799794 0.07534441499999786
orss 1588181.376195631 0.0673570049999995
coc 1827350.0453149527 0.07257541499999931


 22%|██████████████████▋                                                                 | 2/9 [01:58<09:25, 80.77s/it]

...

 78%|█████████████████████████████████████████████████████████████████▎                  | 7/9 [02:37<00:34, 17.25s/it]

schools.csv
random 5895761838.048597 0.019081374999997534
k-means++ 6040321029.971743 0.02355641999999989
orss 6109999678.367918 0.021839594999998723
coc 6337495450.436213 0.023498739999996587


 89%|██████████████████████████████████████████████████████████████████████████▋         | 8/9 [02:39<00:12, 12.61s/it]

Wine.csv
random 1000744.5665473668 0.027716165000002492
k-means++ 1006911.7779631969 0.024208184999999105
orss 990114.7305133507 0.026345010000002844
coc 1037793.4601543492 0.02640023999999812


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [02:41<00:00, 17.96s/it]
